# [05] 모델 만들기3 - PredConv

`PredConv`는 `BaseConv`와 `AuxConv`를 거쳐 얻은 여러 스케일의 featuremap으로부터 bounding box의 좌표들과 class들에 대한 score를 예측하는 모듈입니다. `PredConv`의 예측을 바탕으로 학습을 진행하고, 학습 이후에 실제 활용하는 과정에서도 `PredConv`의 출력값을 이용하게 됩니다.

---------------
`PredConv` 는 이전의 단계에서 얻었던 각 featuemap을 Conv Layer들을 통과시켜 다음을 얻습니다.

- featuremap크기 x bounding box의 좌표 4개의 값 x box개수
- class 정보에 해당하는 1개의 값 x class의 개수

각 feature map에서 예측하는 box의 개수는 미리 정해야 하는 하이퍼파라미터(Hyperparameter)에 해당하는 값입니다. 아래는 (5x5x256)의 dimension의 featuremap을 입력했을 때 `PredConv`의 출력값에 대한 예시입니다. featuremap의 각 위치로부터 box의 좌표 4개의 값과 각 위치에서 예측할 box의 개수 6개를 곱한 24개의 값을 얻는다는 것을 표현하고 있습니다. Class정보의 경우 각 위치에서 box마다 매겨주기 때문에 box의 개수 6개와 class 개수를 곱한 값을 얻게 됩니다.

예를 들어, class 개수가 20개일 때, 최종적으로 얻게 되는 출력값의 차원은 다음과 같습니다.
- Loc(박스예측) : (5x5) x (4) x (6)
- Cls(클래스예측) : (5x5) x (20) x (6)

- [참고] SSD(Single Shot Multibox Detector) 논문 : https://arxiv.org/pdf/1512.02325.pdf

In [ ]:
from IPython.display import HTML, display

# Image from https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection
display(HTML("<img src='img/[05]predconv.png'>"))

-------------------
## [Task 1] `PredConv`로 Box좌표와 Class Score 얻기

`PredConv`에서 박스의 좌표와 클래스 점수를 얻는 방법에 대해 실습해 보도록 하겠습니다. 먼저, 아래의 코드를 실행하여 우리가 `PredConv`에 입력할 6개의 featuremap을 `BaseConv`와 `AuxConv`를 거쳐 얻습니다.

In [1]:
import torch
from materials.det_modules import BaseConvolution, AuxConvolution

baseconv = BaseConvolution()
auxconv = AuxConvolution()

sample = torch.randn(4, 3, 300, 300)
mid, end = baseconv(sample)
features_a, features_b, features_c , features_d = auxconv(end)

print('\nmid shape {}'.format(mid.shape))
print('end shape {}'.format(end.shape))
print('map a shape : {}'.format(features_a.shape))
print('map b shape : {}'.format(features_b.shape))
print('map c shape : {}'.format(features_c.shape))
print('map d shape : {}'.format(features_d.shape))

Loaded pretrained weights for efficientnet-b0

mid shape torch.Size([4, 112, 18, 18])
end shape torch.Size([4, 1280, 9, 9])
map a shape : torch.Size([4, 512, 9, 9])
map b shape : torch.Size([4, 256, 5, 5])
map c shape : torch.Size([4, 256, 3, 3])
map d shape : torch.Size([4, 256, 1, 1])


### ToDo : PredConv 연습하기

먼저, 아래의 `predconv_practice`함수를 완성하여 5x5의 feature map에서 box와 class정보를 얻어봅시다.. 필요한 구성은 다음과 같습니다.

- 예측할 box의 개수 : 6개

- `location_conv` : 3x3 conv를 사용합니다. 5x5의 shape은 유지합니다.

- `class_conv` : 3x3 conv를 사용합니다. 5x5의 shape은 유지합니다.

Feature map의 각 위치마다 예측을 해야 하려면 출력값의 shape이 어떤 형태를 가져야 하는지 고민해 아래 코드를 완성해 봅시다.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def predconv_practice(features_a):
    num_boxes = 6
    class_num = 20
    
    # [ToDo]: loc_conv를 구성합니다.
    loc_conv = ??????????????????????????????????????
    
    # [ToDo]: cls_conv를 구성합니다.
    cls_conv = ??????????????????????????????????????
    
    loc_pred = loc_conv(features_a)
    cls_pred = cls_conv(features_a)
    
    return loc_pred, cls_pred

example = torch.randn(4, 512, 9, 9)
loc_pred, cls_pred = predconv_practice(example)

print('Box Prediction의 차원: {}'.format(loc_pred.shape))
print('Class Score Prediction의 차원: {}'.format(cls_pred.shape))


Box Prediction의 차원: torch.Size([4, 24, 9, 9])
Class Score Prediction의 차원: torch.Size([4, 120, 9, 9])


-------------------
## [Task 2] Box 기준으로 정렬하기

우리가 최종적으로 얻고자 하는 것은 객체(Object)에 대한 Bounding Box와 그에 대한 Class 정보라는 것을 기억해 봅시다. 예측값들을 Box에 대해 표현함으로써 학습과정에서 이후의 Loss 계산과 실제 탐지(Detection)과정에서 필요한 Box들을 추출해내는 작업을 보다 손쉽게 할 수 있습니다.

이번 Task에서는 앞에서 얻은 Box와 Class에 대한 예측값을

- Loc Pred : (box idx, 4x박스개수)
- Cls Pred : (box idx, class 개수)

의 차원을 가지는 함수를 만들어 봅시다.

### ToDo : Box 기준으로 Dimension 바꾸기

1. Tensor의 축을 바꿉니다.

    - (B, 4 x 각 위치의 box 개수, H, W) --> (B, H, W, 4x각 위치의 box 개수)

    - (B, num_class x 각 위치의 box개수, H, W) --> (B, H, W, num_class x 각 위치의 box 개수)

2. Box 기준으로 Shape을 바꿉니다.
    - (B, Box 개수, 4)
    - (B, Box 개수, class_num)

In [ ]:
import torch

loc_example = torch.zeros_like(loc_pred)
cls_example = torch.zeros_like(cls_pred)


def box_align(tensor, mode='loc'):
    num_classes = 20
    
    # batch_size(B)를 저장합니다.
    batch_size = tensor.size(0)
    
    # [ToDo]: (B, box정보, H, W) --> (B, H, W, box정보)
    tensor = tensor.permute(?, ?, ?, ?)
    
    # tensor를 permute된 상태로 확정합니다.
    tensor = tensor.contiguous()
    
    if mode == 'loc':
        # tensor를 (B, Box개수, 4)의 형태로 바꿉니다.
        tensor = tensor.view(?, ?, ?)
        
    elif mode == 'cls':
        # [ToDo]: tensor를 (B, Box개수, num_classes)의 형태로 바꿉니다.
        tensor = tensor.view(?, ?, ?)
    
    return tensor

loc_reshaped = box_align(loc_example, mode='loc')
cls_reshaped = box_align(cls_example, mode='cls')

print('Box Prediction의 차원: {}'.format(loc_reshaped.shape))
print('Class Score Prediction의 차원: {}'.format(cls_reshaped.shape))


------------
## [Task 3] PredictionConvolution Class

`PredConv`도 이전 실습의 `nn.Module` Class로 만들어봅시다.

### ToDo: nn.Module Class로 통합하기

`__init__()` 함수를 완성합니다. 각 featuremap마다 Box Location, Class Score를 예측하기 위한 Conv층을 각각 1개씩 정의합니다. 각 featuremap마다 예측을 위한 Box의 개수는 `n_boxes`로 미리 지정되어 있습니다.

`_box_align` 이전 Task에서 완성한 box_align 함수를 사용합니다.

`forward` 함수를 완성합니다.
- 각 featuremap으로부터 Box Location과 Class Score를 예측합니다.
- _box_align을 거쳐 box 의 차원을 조정합니다.
- Location끼리, Score끼리 예측치들을 concatenation해서 출력합니다.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class PredictionConvolutions(nn.Module):
    def __init__(self, n_classes=20, use_bias=False):
        super(PredictionConvolutions, self).__init__()
        self.n_classes = n_classes
        
        # 각 featuremap에서 예측할 box의 개수를 지정합니다.
        n_boxes = {'mid': 6, 'end': 6, 'a': 6,
                   'b': 6, 'c': 4, 'd': 4}
        
        # box의 좌표를 예측할 conv layer를 구성합니다.
        self.loc_mid = nn.Conv2d(112, n_boxes['mid'] * 4, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.loc_end = nn.Conv2d(1280, n_boxes['end'] * 4, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.loc_a = nn.Conv2d(512, n_boxes['a'] * 4, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.loc_b = nn.Conv2d(256, n_boxes['b'] * 4, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.loc_c = nn.Conv2d(256, n_boxes['c'] * 4, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.loc_d = nn.Conv2d(256, n_boxes['c'] * 4, kernel_size=3, stride=1, padding=1, bias=use_bias)
        
        # box의 class를 예측할 conv layer를 예측합니다.
        self.cls_mid = nn.Conv2d(112, n_boxes['mid'] * n_classes, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.cls_end = nn.Conv2d(1280, n_boxes['end'] * n_classes, kernel_size=3, stride=1, padding=1, bias=use_bias)  
        self.cls_a = nn.Conv2d(512, n_boxes['a'] * n_classes, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.cls_b = nn.Conv2d(256, n_boxes['b'] * n_classes, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.cls_c = nn.Conv2d(256, n_boxes['c'] * n_classes, kernel_size=3, stride=1, padding=1, bias=use_bias)
        self.cls_d = nn.Conv2d(256, n_boxes['d'] * n_classes, kernel_size=3, stride=1, padding=1, bias=use_bias)
        
        self.init__conv2d(use_bias)
        
    def init__conv2d(self, use_bias=False):
        for c in self.children():
            if isinstance(c, nn.Conv2d):
                nn.init.xavier_uniform_(c.weight)
                if use_bias:
                    nn.init.constant_(c.bias, 0.)
                    
                    
    def forward(self, mid, end, a, b, c, d):
        
        # [ToDo]: box 정보를 예측합니다.
        l_mid = ???????????????????
        l_end = ???????????????????
        l_a = ???????????????????
        l_b = ???????????????????
        l_c = ???????????????????
        l_d = ???????????????????
        
        # [ToDo]: self._box_align을 사용해 Box의 Align을 맞춰좁니다.
        l_mid = ???????????????????
        l_end = ???????????????????
        l_a = ???????????????????
        l_b = ???????????????????
        l_c = s???????????????????
        l_d = ???????????????????

        # [ToDo]: box의 class를 예측합니다.
        c_mid = ???????????????????
        c_end = ???????????????????
        c_a = ???????????????????
        c_b = ???????????????????
        c_c = s???????????????????
        c_d = ???????????????????
        
        # [ToDo]: self._box_align을 사용해 Box의 Align을 맞춰좁니다.
        c_mid = ???????????????????
        c_end = ???????????????????
        c_a = ???????????????????
        c_b = ???????????????????
        c_c = ???????????????????
        c_d = ???????????????????
        
        # box좌표, class값을 각각 하나의 tensor로 구성합니다.
        locs = torch.cat([l_mid, l_end, l_a, l_b, l_c, l_d], dim=1)
        cls_scores = torch.cat([c_mid, c_end, c_a, c_b, c_c, c_d], dim=1)
        
        return locs, cls_scores
    
    def _box_align(self, tensor, mode='loc'):
        batch_size = tensor.size(0)
        tensor = tensor.permute(0, 2, 3, 1).contiguous()
        
        if mode == 'loc':
            tensor = tensor.view(batch_size, -1, 4)
        
        elif mode == 'cls':
            tensor = tensor.view(batch_size, -1, self.n_classes)
            
        return tensor

        

In [5]:
predcovtest = PredictionConvolutions(n_classes=20)
locs, cls_scores = predcovtest(mid, end, features_a, features_b, features_c, features_d)

print('Location 예측의 Shape: {}'.format(locs.shape))
print('Clsss Score 예측의 Shape: {}'.format(cls_scores.shape))

Location 예측의 Shape: torch.Size([4, 3106, 4])
Clsss Score 예측의 Shape: torch.Size([4, 3106, 20])


---------
### <생각해 봅시다>

- 각 featuremap마다 예측하는 Box의 개수의 증감에 따른 장단점은 무엇일까요?
- 왜 `PredConv`에서 예측을 위해 3x3 conv를 사용할 때 출력 featuremap의 크기를 유지할까요?

------------